In [52]:
import numpy as np
import os
import glob
from PIL import Image
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [41]:
def load_dataset():
    dataset = []
    for path in glob.glob("./buty/*"):
        img = Image.open(path)
        class_name = os.path.basename(path).split()[0]
        dataset.append((class_name, img))
    return dataset

In [42]:
png_dataset = load_dataset()

In [47]:
def dataset_img_to_vector(dataset):
    labels = []
    images = []
    
    for class_name, img in dataset:
        vec_img = np.asarray(img)
        labels.append(class_name)
        images.append(vec_img)
        
    return labels, images

In [48]:
labels, images = dataset_img_to_vector(png_dataset)